In [1]:
!pip install "ragatouille<0.0.10" langchain langchain-community
#!pip install "ragatouille<0.0.10" langchain==0.0.330 langchain-community==0.0.1 langchain_core==0.0.28


In [4]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pytrec_eval

In [3]:
from ragatouille import RAGPretrainedModel
from datasets import load_dataset

/tmp/ipython-input-4007455469.py:1: UserWarning: 
********************************************************************************
RAGatouille WARNING: Future Release Notice
--------------------------------------------
RAGatouille version 0.0.10 will be migrating to a PyLate backend 
instead of the current Stanford ColBERT backend.
PyLate is a fully mature, feature-equivalent backend, that greatly facilitates compatibility.
However, please pin version <0.0.10 if you require the Stanford ColBERT backend.
********************************************************************************
  from ragatouille import RAGPretrainedModel


In [5]:
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


artifact.metadata: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


In [6]:
import faiss,torch
print("Kmeans available:", hasattr(faiss, 'Kmeans'))

# 测试 GPU Kmeans（需要 CUDA）
if torch.cuda.is_available():
    kmeans = faiss.Kmeans(768, 1000, gpu=True)
    print("GPU Kmeans initialized successfully!")

Kmeans available: True
GPU Kmeans initialized successfully!


In [7]:
rag = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
ds = load_dataset("izhx/COMP5423-25Fall-HQ-small", "default")
documents = [item['text'] for item in ds['collection']]
first = False
if first:
  rag.index(
      collection=documents,
      index_name="hq_small_collection",
      use_faiss=False,
      max_document_length=256,  # Chunk size
      split_documents=True,  # Auto-split long passages
      document_ids=[item['id'] for item in ds['collection']]  # Optional metadata
  )

README.md: 0.00B [00:00, ?B/s]

validation.jsonl: 0.00B [00:00, ?B/s]

collection.jsonl:   0%|          | 0.00/87.7M [00:00<?, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

train.jsonl: 0.00B [00:00, ?B/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating collection split:   0%|          | 0/144718 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1052 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/12000 [00:00<?, ? examples/s]

In [9]:
drive_path = "/content/drive/MyDrive/hq_small_collection/"
rag = RAGPretrainedModel.from_index(drive_path)

In [11]:
# 从数据集加载查询（validation）
qrels = {
    item['id']: {doc_id: 1 for doc_id in item['supporting_ids']}
    for item in ds['validation']
}

from tqdm.auto import tqdm

retrieved_docs = {}
for item in tqdm(ds['validation'], desc="Processing queries"):
    q = item['text']
    qid= item['id']
    results = rag.search(q, k=10)

    doc = {}
    for i in range(10):
        r = results[i]
        doc_id = r['document_id']
        score = r['score']
        doc_current = {doc_id: (10-i)/10}
        doc.update(doc_current)

    retrieved_docs.update({qid : doc})
metrics = compute_metrics(qrels, retrieved_docs)
print(json.dumps(metrics, indent=2))

Processing queries:   0%|          | 0/1500 [00:00<?, ?it/s]

Loading searcher for index hq_small_collection for the first time... This may take a few seconds
[Nov 29, 20:15:07] #> Loading codec...
[Nov 29, 20:15:10] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


W1129 20:15:10.556000 25635 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W1129 20:15:10.556000 25635 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[Nov 29, 20:17:05] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


W1129 20:17:05.791000 25635 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W1129 20:17:05.791000 25635 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

[Nov 29, 20:18:35] #> Loading codes and residuals...




100%|██████████| 7/7 [00:55<00:00,  7.86s/it]


Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: The second place finisher of the 2011 Gran Premio Santander d'Italia drove for who when he won the 2009 FIA Formula One World Championship?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  1996,  2117,  2173,  3926,  2121,  1997,  1996,  2249,
        12604, 23031,  4203, 11563,  1040,  1005, 13052,  5225,  2005,  2040,
         2043,  2002,  2180,  1996,  2268, 19807,  5675,  2028,  2088,  2528,
         1029,   102], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')



/usr/local/lib/python3.12/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


NameError: name 'compute_metrics' is not defined

In [13]:
import argparse
import json
import time

import pytrec_eval
import pandas as pd


def read_jsonl(file_path):
    data = list()
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    print(f'[{time.asctime()}] Read {len(data)} from {file_path}')
    return data


def compute_metrics(qrels: dict[str, dict[str, int]], results: dict[str, dict[str, float]], k_values=[2, 5, 10]):
    map_string = "map_cut." + ",".join([str(k) for k in k_values])
    ndcg_string = "ndcg_cut." + ",".join([str(k) for k in k_values])
    recall_string = "recall." + ",".join([str(k) for k in k_values])
    precision_string = "P." + ",".join([str(k) for k in k_values])
    # qrels = {str(qid): {str(docid): s for docid, s in v.items()} for qid, v in qrels.items()}
    # results = {str(qid): {str(docid): s for s, docid in v} for qid, v in result_heaps.items()}
    evaluator = pytrec_eval.RelevanceEvaluator(
        qrels, {map_string, ndcg_string, recall_string, precision_string}
    )
    scores_by_query = evaluator.evaluate(results)
    scores = pd.DataFrame.from_dict(scores_by_query.values()).mean()
    metrics = dict()
    for prefix in ('map_cut', 'ndcg_cut', 'recall', 'P'):
        name = 'precision' if prefix == 'P' else prefix.split('_')[0]
        for k in k_values:
            metrics[f'{name}_at_{k}'] = scores[f'{prefix}_{k}']
    return metrics

def generate_gold_and_pred_files(queries, rag_results, id_to_supporting_ids):
    gold_data = []
    pred_data = []

    for i, query in enumerate(queries):
        query_id = f"query_{i}" # Assign a unique ID to each query

        # Generate gold data
        # Assuming you have a way to get the true supporting_ids for each query
        # For this example, I'll use the provided id_to_supporting_ids,
        # but in a real evaluation, this would come from your gold standard
        gold_entry = {
            "id": query_id,
            "query": query,
            "supporting_ids": [], # You would need to populate this with the correct supporting IDs
        }
        # Since we don't have a direct mapping from query to supporting_ids in the provided context,
        # I'll leave this empty for now. You'll need to adapt this based on your gold data format.
        # However, if the query ID in the loaded dataset corresponds to a question ID,
        # and the supporting_ids in the dataset are the ground truth, you can use that.
        # Let's assume the document_id in the rag_results is the question ID
        # and use id_to_supporting_ids based on the top retrieved document's ID as a placeholder.
        # This is likely not correct for a real evaluation, but demonstrates the structure.
        if rag_results and len(rag_results) > i and rag_results[i]:
             gold_entry['supporting_ids'] = id_to_supporting_ids.get(rag_results[i][0]['document_id'], [])


        gold_data.append(gold_entry)

        # Generate predicted data
        pred_entry = {
            "id": query_id,
            "query": query,
            "retrieved_docs": [[r['document_id'], r['score']] for r in rag_results[i]] # Use retrieved document IDs and scores
        }
        pred_data.append(pred_entry)

    gold_file_path = "gold.jsonl"
    pred_file_path = "pred.jsonl"

    with open(gold_file_path, 'w') as f:
        for entry in gold_data:
            f.write(json.dumps(entry) + '\n')

    with open(pred_file_path, 'w') as f:
        for entry in pred_data:
            f.write(json.dumps(entry) + '\n')

    return gold_file_path, pred_file_path


def main(gold_file, pred_file):
    gold_data = read_jsonl(gold_file)
    pred_data = read_jsonl(pred_file)
    # Create qrels and results dictionaries with string keys
    qrels = {str(i['id']): {str(d): 1 for d in i['supporting_ids']} for i in gold_data}
    results = {str(i['id']): {str(d): s for d, s in i['retrieved_docs']} for i in pred_data}

    # Ensure query IDs match between qrels and results
    qrels_qids = set(qrels.keys())
    results_qids = set(results.keys())

    if qrels_qids != results_qids:
        print(f"Warning: Query IDs in gold and pred files do not match.")
        # Find missing queries
        missing_in_results = list(qrels_qids - results_qids)
        missing_in_qrels = list(results_qids - qrels_qids)

        if missing_in_results:
            print(f"Query IDs in gold but not in pred: {missing_in_results}")
        if missing_in_qrels:
            print(f"Query IDs in pred but not in gold: {missing_in_qrels}")

        # Consider only the intersection of query IDs for evaluation
        common_qids = list(qrels_qids.intersection(results_qids))
        qrels_common = {qid: qrels[qid] for qid in common_qids}
        results_common = {qid: results[qid] for qid in common_qids}

        if not common_qids:
            print("Error: No common query IDs between gold and pred files. Cannot compute metrics.")
            return {}
        metrics = compute_metrics(qrels_common, results_common)
    else:
        metrics = compute_metrics(qrels, results)


    print(json.dumps(metrics, indent=2))
    return metrics


if __name__ == '__main__':
    # This part will not be executed when called from another cell
    # To run from command line, you can uncomment and adjust the paths
    # gold_file = "gold.jsonl" # Replace with your gold file path
    # pred_file = "pred.jsonl" # Replace with your pred file path
    # main(gold_file, pred_file)
    pass

"""
pip install pytrec_eval pandas
"""

'\npip install pytrec_eval pandas\n'

In [14]:
metrics = compute_metrics(qrels, retrieved_docs)
print(json.dumps(metrics, indent=2))

{
  "map_at_2": 0.5455,
  "map_at_5": 0.6502777777777777,
  "map_at_10": 0.6694882275132276,
  "ndcg_at_2": 0.6218716358293263,
  "ndcg_at_5": 0.7358957947515953,
  "ndcg_at_10": 0.7658869998375539,
  "recall_at_2": 0.5816666666666667,
  "recall_at_5": 0.785,
  "recall_at_10": 0.8603333333333333,
  "precision_at_2": 0.5816666666666667,
  "precision_at_5": 0.314,
  "precision_at_10": 0.17206666666666667
}


In [15]:
from sentence_transformers import CrossEncoder
from tqdm import tqdm
import json

# ===== 新增：初始化重排序模型（放在循环外部）=====
# 推荐模型：适用于英文的 'cross-encoder/ms-marco-MiniLM-L-6-v2'
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', max_length=512)

# ===== 新增：构建文档ID到文本的映射字典 =====
# 假设ds['corpus']包含所有文档，每个item有'id'和'text'字段
corpus_dict = {item['id']: item['text'] for item in ds['collection']}

retrieved_docs = {}
for item in tqdm(ds['validation'], desc="Processing queries with Reranking"):
    q = item['text']
    qid = item['id']

    # 1. 初始检索（保持原始k值，通常k>最终需要的数量）
    initial_k = 50  # 建议：初始检索k应大于最终需要的k（如50>10）
    results = rag.search(q, k=initial_k)

    # 2. 准备重排序数据
    candidate_docs = []
    doc_ids = []
    for r in results:
        doc_id = r['document_id']
        candidate_docs.append(corpus_dict[doc_id])
        doc_ids.append(doc_id)

    # 3. 重排序（注意：处理空结果情况）
    if candidate_docs:
        # 生成(query, doc)对
        pairs = [[q, doc] for doc in candidate_docs]

        # 获取重排序分数
        rerank_scores = reranker.predict(pairs)

        # 按重排序分数降序排序
        reranked_results = sorted(
            zip(doc_ids, rerank_scores),
            key=lambda x: x[1],
            reverse=True
        )

        # 4. 取top-k作为最终结果 (k=10)
        top_k = 10
        final_docs = {doc_id: float(score) for doc_id, score in reranked_results[:top_k]}
    else:
        final_docs = {}

    retrieved_docs[qid] = final_docs

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Processing queries with Reranking: 100%|██████████| 1500/1500 [06:20<00:00,  3.95it/s]


{
  "map_at_2": 0.5825,
  "map_at_5": 0.704888888888889,
  "map_at_10": 0.7176530423280422,
  "ndcg_at_2": 0.6544506545442232,
  "ndcg_at_5": 0.7846360027850993,
  "ndcg_at_10": 0.8038823093934236,
  "recall_at_2": 0.615,
  "recall_at_5": 0.846,
  "recall_at_10": 0.8936666666666667,
  "precision_at_2": 0.615,
  "precision_at_5": 0.3383999999999999,
  "precision_at_10": 0.17873333333333336
}


In [16]:
# 评估（保持不变）
qrels = {
    item['id']: {doc_id: 1 for doc_id in item['supporting_ids']}
    for item in ds['validation']
}
metrics = compute_metrics(qrels, retrieved_docs)
print(json.dumps(metrics, indent=2))

{
  "map_at_2": 0.5825,
  "map_at_5": 0.704888888888889,
  "map_at_10": 0.7176530423280422,
  "ndcg_at_2": 0.6544506545442232,
  "ndcg_at_5": 0.7846360027850993,
  "ndcg_at_10": 0.8038823093934236,
  "recall_at_2": 0.615,
  "recall_at_5": 0.846,
  "recall_at_10": 0.8936666666666667,
  "precision_at_2": 0.615,
  "precision_at_5": 0.3383999999999999,
  "precision_at_10": 0.17873333333333336
}
